In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My\ Drive/trash_classification_project
!ls

/content/drive/My Drive/trash_classification_project
DATASET  __MACOSX


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.1.0
Torchvision Version:  0.3.0


In [0]:
!nvidia-smi

Mon Sep 23 03:32:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |   2293MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
data_dir = r'/content/drive/My Drive/trash_classification_project/DATASET'
model_name = 'inception'
num_class = 2
# 16G memory, we can use 48 batch size or more
batch_size = 48
num_epoch = 15
# frag for feature extracting, if true only update reshape parameters,
# else finetune all parameters 
feature_extract = False

In [0]:
def train_model(model, dataloader, criterion, optimizer, num_epoch, is_inception=False):
    # record time
    since = time.time()
    # accuracy history of val mode
    val_acc_hist = []
    # record the best model and accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # epoches circle
    for epoch in range(num_epoch):
        print('Eopch:{}/{}'.format(epoch, num_epoch-1))
        print('-' * 10)

        # each epoch has two mode parse
        for parse in ['train', 'val']:
            if parse == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0
            running_acc = 0.0

            # load data
            for inputs, labels in dataloader[parse]:
                # convert data
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero gradients
                optimizer.zero_grad()
                
                # get the loss and forward, backward only in val mode
                with torch.set_grad_enabled(parse=='train'):
                    # inception v3 has two outputs, in train mode we calculate
                    # the loss by summing all of them, in test we only consider final
                    if parse == 'train' and is_inception == True:
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    # predict the class
                    _, preds = torch.max(outputs, 1)

                    # backward and optimize if in train mode
                    if parse == 'train':
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_acc += torch.sum(preds == labels.detach())
            # epoch statistics
            epoch_loss = running_loss / len(dataloader[parse].dataset)
            epoch_acc = running_acc.double() / len(dataloader[parse].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(parse, epoch_loss, epoch_acc))

            # save the best model parameters, record accuracy history
            if parse == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if parse == 'val':
                val_acc_hist.append(epoch_acc)
        
        if epoch == 20:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.1
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model
    model.load_state_dict(best_model_wts)
    return model, val_acc_hist

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
def initialize_model(model_name, num_class, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == 'resnet':
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_class)
        input_size = 224

    elif model_name == 'alexnet':
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_class)
        input_size = 224

    elif model_name == 'vgg':
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_class)
        input_size = 224

    elif model_name == 'squeezenet':
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1, 1), stride=(1, 1))
        model_ft.num_class = num_class
        input_size = 224

    elif model_name == 'densenet':
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_class)
        input_size = 224

    elif model_name == 'inception':
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_class)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_class)
        input_size = 299

    else:
        print('Invalid model name, exiting...')
        exit()
    
    return model_ft, input_size

# initialize the model
model_ft, input_size = initialize_model(model_name, num_class, feature_extract, use_pretrained=True)

# print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth
100%|██████████| 108857766/108857766 [00:01<00:00, 105624979.38it/s]


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [0]:
# load data
# data augmentation and normalization for training
data_transforms = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val':transforms.Compose([
                              transforms.Resize(input_size),
                              transforms.CenterCrop(input_size),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

print('Initializing Datasets and Dataloaders...')

# create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                           data_transforms[x]) for x in ['train', 'val']}

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True,
                                                   num_workers=4) for x in ['train', 'val']}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Initializing Datasets and Dataloaders...


In [0]:
# send the model to GPU
model_ft = model_ft.to(device)
# Create the Optimizer
# print the parameters passed to the optimizer
parames_to_update = model_ft.parameters()
print('Parameters to learn:')
if feature_extract:
    parames_to_update = []
    for name, parame in model_ft.named_parameters():
        if parame.requires_grad == True:
            parames_to_update.append(parame)
            print('\t', name)
else:
    for name, parame in model_ft.named_parameters():
        if parame.requires_grad == True:
            print('\t', name)

optimizer_ft = optim.SGD(parames_to_update, lr=0.001, momentum=0.9)

In [0]:
# setup loss function
criterion = nn.CrossEntropyLoss()
# train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epoch, is_inception=(model_name=="inception"))

Eopch:0/14
----------
train Loss: 0.3425 Acc: 0.9034
val Loss: 0.1801 Acc: 0.9447

Eopch:1/14
----------
train Loss: 0.2246 Acc: 0.9437
val Loss: 0.1550 Acc: 0.9542

Eopch:2/14
----------
train Loss: 0.1942 Acc: 0.9499
val Loss: 0.1511 Acc: 0.9503

Eopch:3/14
----------
train Loss: 0.1746 Acc: 0.9551
val Loss: 0.2026 Acc: 0.9351

Eopch:4/14
----------
train Loss: 0.1575 Acc: 0.9615
val Loss: 0.1824 Acc: 0.9431

Eopch:5/14
----------
train Loss: 0.1451 Acc: 0.9645
val Loss: 0.1693 Acc: 0.9463

Eopch:6/14
----------
train Loss: 0.1398 Acc: 0.9662
val Loss: 0.1644 Acc: 0.9447

Eopch:7/14
----------
train Loss: 0.1243 Acc: 0.9711
val Loss: 0.2039 Acc: 0.9379

Eopch:8/14
----------
train Loss: 0.1189 Acc: 0.9726
val Loss: 0.1924 Acc: 0.9455

Eopch:9/14
----------
train Loss: 0.1154 Acc: 0.9721
val Loss: 0.2107 Acc: 0.9395

Eopch:10/14
----------
train Loss: 0.1058 Acc: 0.9745
val Loss: 0.2062 Acc: 0.9443

Eopch:11/14
----------
train Loss: 0.1080 Acc: 0.9751
val Loss: 0.2009 Acc: 0.9475

Eo

In [0]:
# Comparison with model trained from scratch
scratch_model,_ = initialize_model(model_name, num_class, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_, scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, 40, is_inception=(model_name=='inception'))

# plot the curve of validation accuracy vs epoch num of 
# transfer learning and model trained from scratch
ohist = []
shist = []

ohist = [h.cpu().numpy() for h in hist]
shist = [h.cpu().numpy() for h in scratch_hist]

plt.title('Validation accuracy vs epoch number')
plt.xlabel('epoch number')
plt.ylabel('validation accuracy')
plt.plot(range(1, num_epoch+1), ohist, label='pretrained')
plt.plot(range(1, num_epoch+1), shist, label='scratch')
plt.ylim((0, 1.))
plt.xticks(np.arange(1, num_epoch +1, 1.0))
plt.legend()
plt.show()